# Importation et manipulation de données spatiales {#sec-chap01}

Dans le chapitre, nous abordons quelques formats d'images ainsi que leur lecture.

::: bloc_package
::: bloc_package-header
::: bloc_package-icon
:::

**Liste des *librairies* utilisées dans ce chapitre**
:::

::: bloc_package-body
-   Pour importer et manipuler des fichiers géographiques :
    -   `sf` pour importer et manipuler des données vectorielles.
    -   `Rasterio` pour importer des images.
    -   `NumPy` pour manipuler des données matricielles.
-   Pour construire des cartes et des graphiques :
    -   `tmap` est certainement le meilleur *package* pour la cartographie.
    -   `Matplotlib` pour construire des graphiques.
:::
:::


In [ ]:
#| echo: false
#| eval: true
#| warning: false
!pip install -q rasterio

## Bases de Quarto... {#sec-010}

Voici comment faire une liste :

-   **texte en gras**
-   *texte en italique*
    -   ***Gras et italique***
-   R^2^ et NO~2~
-   [Petites majuscules]{.smallcaps}
-   Pour un lien Web, [Département de géomatique appliquée](https://www.usherbrooke.ca/geomatique/).

Voici comment intégrer des équations LaTeX. La formule de la distance euclidienne (@eq-DistEuc).
Pour écrire directement une équation dans le texte, il suffit de $E = mc^2$.

$$
 d_{ij} = 2R \cdot \text{ arcsin} \left( \sqrt{\text{sin}^2 \left( \frac{\delta _i - \delta _j}{2} \right) + \text{cos }\delta _i \cdot \text{cos }\delta _j \cdot \text{sin}^2 \left( \frac{\phi _i - \phi _j}{2} \right)} \right)
$$ {#eq-DistLongLat}

$$
 d_{ij} = \sqrt{(x_i-x_j)^2+(y_i-y_j)^2}
$$ {#eq-DistEuc}

-   Intégrer une figure (image) et l'appeler dans le texte (@fig-downloaffromgit). À la @fig-downloaffromgit, ... Notez que la numérotation des figures, des tableaux, des équations est automatique.

![Téléchargement de l'intégralité du livre](https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/introduction/download_github.png){#fig-downloaffromgit width="40%" fig-align="center"}


-   Les références sont au format BibTeX. Par exemple, vous pouvez les télécharger de Google Scholar et les intégrer à la fin du fichier references.bib. Voici comment intégrer des références [@mather2022computer; @richards2022remote]. Selon Ferdinand Boon et Guy Rochon [-@PrecisTeleVol1].



## Importation d'images {#sec-011}

La première étape avant tout traitement est d'accéder à la donnée image pour qu'elle soit manipulée par le programme Python. L'imagerie satellite présente certains défis notamment en raison de la taille parfois très importante des images. Il existe maintenant certaines librairies, comme 🔖[Xarray](https://docs.xarray.dev/en/stable/), qui on cherchées à optimiser la lecture et l'écriture de grandes images. Il est donc conseiller de toujours garder un oeil sur l'espace mémoire occupé par les variables Python reliées aux images. La librairie principale en géomatique qui va nous permettre d'importer (et d'exporter) de l'imagerie est la librairie [GDAL](https://gdal.org) qui rassemble la plupart des formats sous forme de *driver* (ou pilote en français). 

Dans le domaine de la géomatique, il faut prêter attention à trois caractéristiques principales des images:
1. **La matrice des données** elle-même qui contient les valeurs brutes des pixels. Cette matrice sera souvent un cube à trois dimensions. En Python, ce cube sera le plus souvent un objet de la librairie 🔖[NumPy](https://numpy.org/) (voir section).
2. **La dynamique des images** c.à.d le format de stockage des valeurs individuelles (octet, entier, double, etc.). Ce format décide principalement de la résolution radiométrique et des valeurs minimales et maximales supportées. 
3. **La métadonnée** qui va transporter l'information auxiliaire de l'image comme les dimensions et la position de l'image, la date, etc. Cette donnée auxiliaire prendra souvent la forme d'un dictionnaire Python. 

Les différents formats se distinguent principalement sur la manière dont ces trois caractéristiques sont gérées.

### Formats des images

Il existe maintenant de nombreux formats numériques pour la donnée de type image parfois appelé donnée matricielle ou donnée *raster*. La librairie GDAL rassemble la plupart des formats matriciels rencontrés en géomatique (voir 🔖[Raster drivers — GDAL documentation](https://gdal.org/en/latest/drivers/raster/index.html) pour une liste complète).

On peut distinguer deux grandes familles de format:
1. Les formats de type **RVB** issus de l'imagerie numérique grand publique comme 🔖[JPEG](https://gdal.org/en/latest/drivers/raster/jpeg.html#raster-jpeg), [png](https://gdal.org/en/latest/drivers/raster/png.html#raster-png), etc. Ces formats ne supportent généralement que trois bandes au maximum (rouge, vert et bleu) et des valeurs de niveaux de gris entre 0 et 255 (format dit 8 bit).
2. **Les géo-formats** issus des domaines scientifiques ou techniques comme GeoTIFF, HDF5, etc. qui peuvent inclure plus que trois bandes et des dynamiques plus élevées (16 bit ou même float).

Les formats RVB restent très utilisés en Python notamment par les librairies dites de vision par ordinateur (*Computer Vision*) comme OpenCV et sickit-image ainsi que les grandes librairies en apprentissage profond (PyTorch, Tensorflow).  

::: bloc_package
::: bloc_package-header
::: bloc_package-icon
:::

**Installation de gdal dans un système Linux **
:::

::: bloc_package-body
-   Pour installer GDAL :
```
!apt-get update
!apt-get install gdal-bin libgdal-dev
```
:::
:::


#### Formats de type RVB 

Les premiers formats pour de l'imagerie à une bande (monochrome) et à trois bandes (image couleur rouge-vert-bleu) sont issus du domaine des sciences de l'ordinateur. On trouvera, entre autres, les formats pbm, png et jpeg. Ces formats supportent peu de métadonnées et sont placées dans un entête (*header*) très limité. Cependant, ces formats restent très populaires dans le domaine de la vision par ordinateur et sont très utilisés en apprentissage profond en particulier. Pour la lecture des images RVB, on peut utiliser les librairies Rasterio, [PIL](https://he-arc.github.io/livre-python/pillow/index.html) ou [OpenCV](https://docs.opencv.org/4.10.0/index.html).

##### Lecture avec la librairie PIL
La librairie PIL retourne un objet de type `PngImageFile`, l'affichage de l'image se fait directement dans la cellule de sortie.

In [ ]:
#| lst-label: lst-lecture-PIL-PNG
#| lst-cap: Lecture d'une image en format PNG avec PIL
#| eval: true

!wget https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/modis-aqua.PNG -O modis-aqua.PNG
from PIL import Image
img = Image.open('modis-aqua.PNG')
img

##### Lecture avec la librairie OpenCV
La librairie [OpenCV](https://docs.opencv.org/4.10.0/index.html) est aussi très populaire en vision par ordinateur. La fonction `imread` donne directement un objet de type NumPy en sortie.

In [ ]:
#| lst-label: lst-lecture-opencv-PNG
#| lst-cap: Lecture d'une image en format PNG avec OpenCV

!wget https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/modis-aqua.PNG -O modis-aqua.PNG
import cv2
img = cv2.imread('modis-aqua.PNG')
img

##### Lecture avec la librairie RasterIO

Rien ne nous empêche de lire une image de format RVB avec [RasterIO](https://rasterio.readthedocs.io/en/stable/) comme décrit dans (@lst-lecturerasterioPNG). Vous noterez cependant les avertissements concernant l'absence de géoréférence pour ce type d'image.

In [ ]:
#| lst-label: lst-lecturerasterioPNG
#| lst-cap: Lecture d'une image en format PNG avec OpenCV

!wget https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/modis-aqua.PNG -O modis-aqua.PNG
import rasterio
img= rasterio.open('modis-aqua.PNG')
img

#### Le format GeoTiff 

Le format GeoTIFF est une extension du format TIFF (Tagged Image File Format) qui permet d'incorporer des métadonnées géospatiales directement dans un fichier image. Développé initialement par Dr. Niles Ritter au Jet Propulsion Laboratory de la [NASA](https://www.earthdata.nasa.gov/esdis/esco/standards-and-practices/geotiff){target="_blank"} dans les années 1990, GeoTIFF est devenu un standard de facto pour le stockage et l'échange d'images géoréférencées dans les domaines de la télédétection et des systèmes d'information géographique (SIG). Ce format supporte plus que trois bandes aussi longtemps que ces bandes sont de même dimension.

Le format GeoTIFF est très utilisé et est largement supporté par les bibliothèques et logiciels géospatiaux, notamment [GDAL](https://gdal.org) (*Geospatial Data Abstraction Library*), qui offre des capacités de lecture et d'écriture pour ce format. Cette compatibilité étendue a contribué à son adoption généralisée dans la communauté géospatiale.

##### Standardisation par l'OGC 

Le standard GeoTIFF proposé par l'Open Geospatial Consortium (OGC) en 2019 formalise et étend les spécifications originales du format GeoTIFF, offrant une norme robuste pour l'échange d'images géoréférencées. Cette standardisation, connue sous le nom d'OGC GeoTIFF 1.1 [-@OGCGeoTIFF], apporte plusieurs améliorations et clarifications importantes.


#### Le format COG

Une innovation récente dans l'écosystème GeoTIFF est le format *Cloud Optimized GeoTIFF* ([COG](http://cogeo.org/)), conçu pour faciliter l'utilisation de fichiers GeoTIFF hébergés sur des serveurs web HTTP. Le COG permet aux utilisateurs et aux logiciels d'accéder à des parties spécifiques du fichier sans avoir à le télécharger entièrement, ce qui est particulièrement utile pour les applications basées sur le cloud.

### Métadonnées des images 

La manière la plus directe d'accéder à la métadonnée d'une image est d'utiliser les commandes 🔖[`rio info`](https://rasterio.readthedocs.io/en/stable/cli.html#info) de la librairie Rasterio ou `gdalinfo` de la librairie `gdal`. Le résultat est imprimé dans la sortie standard ou sous forme d'un dictionnaire Python.

In [ ]:
#| lst-label: lst-lecturerasterioPNG
#| lst-cap: Lecture d'une image en format PNG avec OpenCV

!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_RGBNIR_of_S2A_MSIL2A_20240625T153941_N0510_R011_T18TYR_20240625T221903.tif -O RGBNIR_of_S2A.tif

!gdalinfo RGBNIR_of_S2A.tif

Le plus simple est d'utiliser la fonction `rio info`:

In [ ]:
#| lst-label: lst-lecturerasterioPNG
#| lst-cap: Lecture d'une image en format PNG avec OpenCV
#| eval: true

!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_RGBNIR_of_S2A_MSIL2A_20240625T153941_N0510_R011_T18TYR_20240625T221903.tif -q -O RGBNIR_of_S2A.tif

!rio info RGBNIR_of_S2A.tif --indent 2 --verbose

## Manipulation des images {#sec-012}

### Manipulation de la matrice de pixels 

La donnée brute de l'image est généralement contenue dans un cube matricielle à trois dimensions (deux dimensions spatiales et une dimension spectrale). Comme exposé précédemment, la librairie dite *"fondationnelle"* pour la manipulation de matrices en Python est [NumPy](https://numpy.org/). Cette librairie contient un nombre très important de fonctionnalités couvrant l'algèbre linéaires, les statistiques, etc. et constitue la fondation de nombreuses librairies (voir (@fig-naturenumpy1))

![La librairie NumPy est le fondement de nombreuses librairies scientifiques (d'après [@NumpyNature]).](https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/41586_2020_2649_Fig2_HTML.png){#fig-naturenumpy1 width="100%" fig-align="center"}

### Information de base

Les deux informations de base à afficher sur une matrice est 1) les dimensions de la matrice et 2) le format de stockage (le type). Pour cela, on peut utiliser le (@lst-numpyshape), le résultat nous informe que la matrice a 3 dimensions et une taille de `(442, 553, 3)` et un type `uint8` qui représente 1 octet (8 bit). Par conséquent, la matrice a `442` lignes, `553` colonnes et `3` canaux ou bandes. Il faut prêter une attention particulière aux valeurs minimales et maximales tolérées par le type de la donnée comme indiqué dans le (@tbl-numpytype) (voir aussi 🔖[Data types — NumPy v2.1 Manual](https://numpy.org/doc/stable/user/basics.types.html)).


In [ ]:
#| lst-label: lst-numpyshape
#| lst-cap: Lecture d'une image en format PNG avec OpenCV
#| eval: true

!wget https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/modis-aqua.PNG --quiet -O modis-aqua.PNG
import cv2
img = cv2.imread('modis-aqua.PNG')
print('Nombre de dimensions: ',img.ndim)
print('Dimensions de la matrice: ',img.shape)
print('Type de la donnée: ',img.dtype)

In [ ]:
#| label: tbl-numpytype
#| tbl-cap: "Type de données de NumPy"
#| eval: true

from IPython.display import Markdown
from tabulate import tabulate
table = [["uint8", "char", 8, 0, 255],
        ["int8", "signed char", 8, -127, +128],
        ["uint16", "unsigned short", 16, 0, -32768, +32767],
        ["int16", "short", 16, 0, 655355]]
Markdown(tabulate(table, headers=["dtype", "Nom", "Taille", "Min", "Max"], tablefmt="pipe"))

::: bloc_aller_loin
::: bloc_aller_loin-header
::: bloc_aller_loin-icon
:::

**Les différents types de données en dans NumPy**
:::

::: bloc_aller_loin-body
Il comprend des références ou des extensions d'une méthode abordée dans une section.
:::
:::

### Découpage et indexation de la matrice

L'indexation et le découpage des matrices dans NumPy sont des techniques essentielles pour manipuler efficacement les données multidimensionnelles en Python, offrant une syntaxe puissante et flexible pour accéder et modifier des sous-ensembles spécifiques d'éléments dans les tableaux. Indexer une matrice consiste à accéder à une valeur dans la matrice pour une position particulière, la syntaxe générale est `matrice[ligne, colonne, bande]`. Les indices commencent à 0 et se termine à la `dimension-1` de l'axe considéré. 

![Téléchargement de l'intégralité du livre](https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/41586_2020_2649_Fig1_HTML.png){#fig-naturenumpy2 width="100%" fig-align="center"}

Le découpage (ou *slicing* en anglais) consiste à produire une nouvelle matrice qui est un sous-ensemble de la matrice d'origine. Un découpage se fait avec le symbole ':', la syntaxe générale pour définir un découpage est `[début:fin:pas]`. Si on ne spécifie pas `début` ou `fin` alors les valeurs 0 ou `dimension-1` sont considérées implicitement. Quelques exemples:
* choisir un pixel en particulier avec toutes les bandes: `matrice[1,1,:]`
* choisir la colonne 2: `matrice[:,2,:]`

La syntaxe de base pour le découpage (*slicing*) des tableaux NumPy repose sur l'utilisation des deux-points (`:`) à l'intérieur des crochets d'indexation. Cette notation permet de sélectionner des plages d'éléments de manière concise et intuitive.
La structure générale du découpage est `matrice[start:stop:step]`, où :
1. `start` représente l'index de départ (inclus)
2. `stop` indique l'index de fin (exclu)
3. `step` définit l'intervalle entre chaque élément sélectionné

Si l'un de ces paramètres est omis, NumPy utilise des valeurs par défaut : 0 pour `start`, la taille du tableau pour `stop`, et 1 pour `step`. Par exemple, pour un tableau unidimensionnel `array`, on peut extraire les éléments du deuxième au quatrième avec `array[1:4]`. Pour sélectionner tous les éléments à partir du troisième, on utiliserait `array[2:]`. Cette syntaxe s'applique également aux tableaux multidimensionnels, où chaque dimension est séparée par une virgule. Ainsi, pour une matrice 2D m, m[0:2, 1:3] sélectionnerait une sous-matrice 2x2 composée des deux premières lignes et des deuxième et troisième colonnes. L'indexation négative est également supportée, permettant de compter à partir de la fin du tableau. Par exemple, a[-3:] sélectionnerait les trois derniers éléments d'un tableau.

In [ ]:
#| eval: true

import cv2
img = cv2.imread('modis-aqua.PNG')
img_col = img[:,1,:]
print('Nombre de dimensions: ',img_col.ndim)
print('Dimensions de la matrice: ',img_col.shape)

::: bloc_aller_loin
::: bloc_aller_loin-header
::: bloc_aller_loin-icon
:::

**Une vue versus une copie**
:::

::: bloc_aller_loin-body
Avec Numpy, les manipulations peuvent créer des vues ou des copies. Une vue est une simple représentation de la même donnée originale alors qu'une copie est un nouvel espace mémoire.

Par défaut, un découpage créé une vue.

On peut vérifier si l'espace mémoire est partagé avec `np.shares_memory(arr, slice_arr)`.

On peut toujours forcer une copie avec la méthode `copy()`
:::
:::


### Mosaïquage, masquage et découpage 

### Changement de projection cartographique 

### Recalage d'images et co-registration 

## Données en géoscience {#sec-013}

Calibration, unités, données manquantes, données éparses.

netcdf, xarray, GRIB.

Données météos, exemple avec SWOT.

## Importation de données vectorielles {#sec-012}

### Importation d'un fichier *shapefile* {#sec-0121}

### Importation d'une couche dans un *GeoPackage* {#sec-0122}

### Importation d'une couche dans une *geodatabase* d'ESRI {#sec-0123}

### Importation d'un fichier *shapefile* {#sec-0124}

## Manipulation de données vectorielles {#sec-014}

### Requêtes attributaires 

## Quiz de révision du chapitre {#sec-015}

## Exercices de révision {#sec-016}

::: bloc_exercice
::: bloc_exercice-header
::: bloc_exercice-icon
:::

**Exercice 1.** À compléter
:::

::: bloc_exercice-body

In [ ]:
#| echo: true
#| warning: false
#| eval: false

# ...
# à compléter

Correction à la [section @sec-08011].
:::
:::

::: bloc_exercice
::: bloc_exercice-header
::: bloc_exercice-icon
:::

**Exercice 2.** À compléter
:::

::: bloc_exercice-body

In [ ]:
#| echo: true
#| warning: false
#| eval: false

# ...
# à compléter

Correction à la [section @sec-08012].
:::
:::

::: bloc_exercice
::: bloc_exercice-header
::: bloc_exercice-icon
:::

**Exercice 3.** À compléter
:::

::: bloc_exercice-body

In [ ]:
#| echo: true
#| warning: false
#| eval: false

# ...
# à compléter

Correction à la [section @sec-08013].
:::
:::